# 시계열 다루기
1. 타임스팸트: 특정 시점 (e.g. 2022년 5월 4일 오전 7시)
2. 시간 간격/기간: 특정 시작점과 종료점 사이 시간의 길이 
3. time delta / duration (시간 델타/지속 기간): 정확한 시간의 길이 (e.g. 23.33초의 시간)

### 기본 파이썬 날짜와 시간: datetime / dateutil 

In [1]:
from datetime import datetime
datetime(year = 2015, month = 7, day = 4)

datetime.datetime(2015, 7, 4, 0, 0)

In [3]:
from dateutil import parser 
date = parser.parse('4th of July, 2015')
date

datetime.datetime(2015, 7, 4, 0, 0)

In [4]:
date.strftime('%A')

'Saturday'

In [8]:
## numpy로!

import numpy as np

date = np.array('2022-05-01', dtype=np.datetime64)
date # 이제 연산이 빨라짐   

array('2022-05-01', dtype='datetime64[D]')

In [11]:
print(date + np.array(12)) # 12일 더한거
print(date + np.arange(12)) # date 부터 12개 쭈욱!

2022-05-13
['2022-05-01' '2022-05-02' '2022-05-03' '2022-05-04' '2022-05-05'
 '2022-05-06' '2022-05-07' '2022-05-08' '2022-05-09' '2022-05-10'
 '2022-05-11' '2022-05-12']
